In [1]:
%%bash
#!/bin/bash

CMSSW_VER="CMSSW_8_1_0"
KERNEL_NAME="higgs-combine-tutorial"

set -e
# Get the CMSSW libraries (specifically ROOT)
SCRAM_ARCH=$(ls -d /cvmfs/cms.cern.ch/$(/cvmfs/cms.cern.ch/common/cmsos)*/cms/cmssw/${CMSSW_VER} | tail -n 1 | awk -F / '{ print $4 }')
export SCRAM_ARCH
source /cvmfs/cms.cern.ch/cmsset_default.sh
#if [ ! -d cmssw-env ]; then
#    scramv1 project --name cmssw-env CMSSW $CMSSW_VER 
#fi

# Now, install additional python packages
cd ~/CombineTutorial/CMSSW_8_1_0/
eval `scramv1 runtime -sh`
cd src

echo "Using the following python: $(which python2)"
set +e
if ! python2 -c "import pip"; then
    wget https://bootstrap.pypa.io/get-pip.py
    python2 get-pip.py --user
fi
if [ $? -ne 0 ]; then
    wget https://bootstrap.pypa.io/get-pip.py
    python2 get-pip.py --user
fi
if [ ! -d venv ]; then
    # We need to bring the "system" packages, which is the CMSSW env
    # in this case
    virtualenv --python="$(which python2)" venv
    #  --system-site-packages 
fi
source venv/bin/activate
python2 -m pip install ipykernel

cd ..

# Make a wrapper script to load CMSSW python
cat << 'EOF' > bin/python_wrapper.sh
#!/bin/bash
DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" && pwd )"
source /cvmfs/cms.cern.ch/cmsset_default.sh
pushd $DIR
eval `scramv1 runtime -sh`
pushd ../src
source venv/bin/activate
## Delete this absolutely ancient jupyter component
PYTHONPATH=$(echo $PYTHONPATH | sed 's,/cvmfs/cms.cern.ch/slc7_amd64_gcc530/external/py2-ipython/3.1.0-giojec4/lib/python2.7/site-packages,,')
CMSSW_PYTHON="$(which python2)"
popd ; popd
exec "$CMSSW_PYTHON" "$@"
EOF
chmod +x bin/python_wrapper.sh

# Create the kernel
mkdir -p "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME"
cat << EOF > "$HOME/.local/share/jupyter/kernels/$KERNEL_NAME/kernel.json"
{
 "display_name": "$KERNEL_NAME", 
 "language": "python", 
 "argv": [
  "$PWD/bin/python_wrapper.sh", 
  "-m", 
  "ipykernel_launcher", 
  "-f", 
  "{connection_file}"
 ]
}
EOF

# Report OK
echo "Loaded $CMSSW_VERSION into $KERNEL_NAME!"


Using the following python: /cvmfs/cms.cern.ch/slc7_amd64_gcc530/cms/cmssw/CMSSW_8_1_0/external/slc7_amd64_gcc530/bin/python2
New python executable in /home/dsperka/CombineTutorial/CMSSW_8_1_0/src/venv/bin/python2
Also creating executable in /home/dsperka/CombineTutorial/CMSSW_8_1_0/src/venv/bin/python
Installing setuptools, pip, wheel...done.
Running virtualenv with interpreter /cvmfs/cms.cern.ch/slc7_amd64_gcc530/cms/cmssw/CMSSW_8_1_0/external/slc7_amd64_gcc530/bin/python2
  Using cached https://files.pythonhosted.org/packages/00/47/764e4fa1b1b89598426b8d79b1c4fbe8042432621b0f8e1991aeb3c24806/ipykernel-4.10.0-py2-none-any.whl
  Using cached https://files.pythonhosted.org/packages/94/dd/fe6c4d683b09eb05342bd2816b7779663f71762b4fa9c2d5203d35d17354/jupyter_client-5.2.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b0/88/d996ab8be22cea1eaa18baee3678a11265e18cf09974728d683c51102148/ipython-5.8.0-py2-none-any.whl
  Using cached https://files.pythonhosted.org/p

perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = (unset),
	LC_ALL = (unset),
	LANG = "C.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to the standard locale ("C").
matplotlib 1.5.2 requires cycler, which is not installed.
